## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-25-07-49-23 +0000,bbc,Government considers financial support for JLR...,https://www.bbc.com/news/articles/c62nv0xx32go...
1,2025-09-25-07-49-00 +0000,wsj,Swiss Central Bank on Hold as Tariffs Threaten...,https://www.wsj.com/articles/swiss-central-ban...
2,2025-09-25-07-46-25 +0000,bbc,Labour MPs are privately urging me to challeng...,https://www.bbc.com/news/articles/c7v1m873mjyo...
3,2025-09-25-07-40-44 +0000,nyt,Kashmir’s Apple Harvest Rots After Landslide B...,https://www.nytimes.com/2025/09/25/world/asia/...
4,2025-09-25-07-36-33 +0000,bbc,Co-op says cyber-attack cost it £206m in sales,https://www.bbc.com/news/articles/ckgq9dke4e5o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2352/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
259,trump,57
356,ice,15
224,new,14
79,kimmel,13
360,shooting,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
311,2025-09-24-09-30-00 +0000,wsj,Stephen Miran set out his case for lower inter...,https://www.wsj.com/economy/central-banking/st...,107
249,2025-09-24-14-05-20 +0000,nypost,Trump threatens to sue ABC over Jimmy Kimmel’s...,https://nypost.com/2025/09/24/us-news/trump-th...,106
132,2025-09-24-21-39-41 +0000,wapo,President Donald Trump wrote on TruthSocial th...,https://www.washingtonpost.com,102
101,2025-09-24-22-55-47 +0000,nypost,Trump’s pro-Ukraine shift a ‘strategic move’ b...,https://nypost.com/2025/09/24/world-news/trump...,98
119,2025-09-24-22-15-02 +0000,latimes,Trump's attacks on Kimmel and ABC put him at o...,https://www.latimes.com/politics/story/2025-09...,91


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
311,107,2025-09-24-09-30-00 +0000,wsj,Stephen Miran set out his case for lower inter...,https://www.wsj.com/economy/central-banking/st...
91,65,2025-09-24-23-28-04 +0000,nyt,Dallas ICE Facility Shooting Leaves One Detain...,https://www.nytimes.com/2025/09/24/us/dallas-i...
231,60,2025-09-24-15-46-02 +0000,bbc,Jimmy Kimmel calls out 'anti-American' threats...,https://www.bbc.com/news/articles/c9dx6dlelw8o...
40,44,2025-09-25-02-51-33 +0000,nypost,"Mom who lost daughter, 15, to suicide after on...",https://nypost.com/2025/09/24/world-news/mom-w...
269,42,2025-09-24-12-19-49 +0000,bbc,Typhoon Ragasa makes landfall in China after 1...,https://www.bbc.com/news/articles/clyd580lx3mo...
238,42,2025-09-24-15-06-31 +0000,bbc,Israeli forces near Gaza City centre as Hamas ...,https://www.bbc.com/news/articles/c4gk9dreldxo...
101,41,2025-09-24-22-55-47 +0000,nypost,Trump’s pro-Ukraine shift a ‘strategic move’ b...,https://nypost.com/2025/09/24/world-news/trump...
52,37,2025-09-25-01-34-30 +0000,nypost,Trans boyfriend of Charlie Kirk suspect Tyler ...,https://nypost.com/2025/09/24/us-news/trans-bo...
64,34,2025-09-25-01-00-00 +0000,wsj,RFK Jr.’s team wanted to tout an autism therap...,https://www.wsj.com/health/healthcare/tylenol-...
204,34,2025-09-24-17-33-28 +0000,nypost,Zohran Mamdani’s dominant lead in latest NYC m...,https://nypost.com/2025/09/24/us-news/zohran-m...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
